# Data Preprecessing

## Import Required Libraries

In [77]:
import os
import librosa
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

## Functions to Convert MP3 Files to Mel Spectrograms

In [79]:
# Function to convert MP3 files to mel spectrograms
def mp3_to_spectrogram(file_path, n_mels = 128, duration = 5, sr = 22050):
    # Load the MP3 file
    y, _ = librosa.load(file_path, sr = sr, duration = duration)

    # Use libraosa to compute the mel spectrogram
    mels = librosa.feature.melspectrogram(y = y, sr = sr, n_mels = n_mels)

    # Convert the mel spectrogram to decibel scale
    log_mels = librosa.power_to_db(mels)

    # Return the log-mel spectrogram
    return log_mels

# Function to save the mel spectragrams as numpy arrays
def save_spectogram_as_npy(file_path, output_path):
    # Convert the MP3 file to mel spectrogram
    mel = mp3_to_spectrogram(file_path)

    # Save the spectrogram as a numpy array
    np.save(output_path, mel)

## Convert MP3 Files to Mel Spectrograms

In [80]:
# Loop through ecah bird class directory
for bird in bird_classes:
    # Find directories for each bird class
    bird_directory = os.path.join("../data/raw/", bird)

    # Establish output directory for processed data
    output_dir = os.path.join("../data/processed/", bird)
    os.makedirs(output_dir, exist_ok = True)
    
    # Loop through each MP3 file in the bird class directory
    for filename in os.listdir(bird_directory):

        # Check if the file is an MP3 file
        if filename.endswith(".mp3"):
            # Construct the full file path
            file_path = os.path.join(bird_directory, filename)

            # Construct the full outut path
            output_path = os.path.join(output_dir, filename.replace(".mp3", ".npy"))

            # Save the mel spectrogram as a numpy array
            save_spectogram_as_npy(file_path, output_path)

# Inspect Spectrogram Sizes

In [81]:
# Investigate the shapes of the spectorgams
def inspect_spectogram_shapes(data_directory = "../data/processed/"):
    # Initialize a list to store the shapes of the spectrograms
    shape_list = []

    # Loop through ecah bird direcotry
    for bird in os.listdir(data_directory):
        bird_directory = os.path.join(data_directory, bird)

        # Check if directory exists
        if not os.path.isdir(bird_directory):
            continue

        # Loop through each numpy file
        for file_name in os.listdir(bird_directory):

            # Check if the file is a numpy file
            if file_name.endswith(".npy"):
                file_path = os.path.join(bird_directory, file_name)

                # Load the numpy file
                spectogram = np.load(file_path)

                # Get the file shape and append it to the list
                shape = spectogram.shape
                shape_list.append(shape)

    # Return the list of spectogram shapes
    return shape_list

In [82]:
# Inspect shapes of ALL spectrograms regardless of bird class
spectogram_shapes = inspect_spectogram_shapes()

# Initialize lsits to store heights and widths
heights = []
widths = []

# Loop through the shapes and extract hieghts and widths
for shape in spectogram_shapes:
    heights.append(shape[0])
    widths.append(shape[1])

print("Summary:")
print(f"\tMin width: {min(widths)}")
print(f"\tMax width: {max(widths)}")
print(f"\tMean width: {sum(widths) / len(widths):.2f}")
print(f"\tFixed height (n_mels): {set(heights)}")

print(spectogram_shapes)

Summary:
	Min width: 156
	Max width: 216
	Mean width: 214.16
	Fixed height (n_mels): {128}
[(128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 156), (128, 216), (128, 193), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216), (128, 216)]


## Pad or Crop Spectrograms to a Fixed Size

In [83]:
# Ensure spectrgrams are the same size
def pad_or_crop(spectrogram, target_width = 216):
    # Get spectrogram width
    _, width = spectrogram.shape

    # Check if the width is greater than the target width
    if width > target_width:
        # Crop the spectrogram at the center
        start = (width - target_width) // 2

        # Ensure the spectrogram is cropped correctly
        spectrogram = spectrogram[:, start : start + target_width]

    # Check if the width is less than the target width
    elif width < target_width:
        # Determine how much to pad
        pad_width = target_width - width

        # Pad the spectrogram with zeros
        spectrogram = np.pad(spectrogram, ((0, 0), (0, pad_width)), mode = "constant")
    
    # Return the padded or cropped spectrogram
    return spectrogram

In [84]:
# Apply the paddin/cropping function to all spectrograms
def standardize_all_spectograms(data_directory = "../data/processed/", target_width = 216):

    # Loop through each processed bird directory
    for bird in os.listdir(data_directory):
        bird_directory = os.path.join(data_directory, bird)
        if not os.path.isdir(bird_directory):
            continue

        # Loop through each numpy file in the processed bird directory
        for file_name in os.listdir(bird_directory):

            # Check if the file in a numpy file
            if file_name.endswith(".npy"):
                # Get the full file path
                file_path = os.path.join(bird_directory, file_name)
                spectrogram = np.load(file_path)

                # Standardize the spectrogram width
                spectrogram_fixed = pad_or_crop(spectrogram, target_width = target_width)

                # Overwrite the file
                np.save(file_path, spectrogram_fixed)

                print(f"Processed: {file_path}:\n\tBEFORE: {spectrogram.shape} \n\tAFTER: {spectrogram_fixed.shape}")

In [85]:
standardize_all_spectograms("../data/processed", target_width = 216)

Processed: ../data/processed\bald_eagle\bald_eagle_1.npy:
	BEFORE: (128, 216) 
	AFTER: (128, 216)
Processed: ../data/processed\bald_eagle\bald_eagle_10.npy:
	BEFORE: (128, 216) 
	AFTER: (128, 216)
Processed: ../data/processed\bald_eagle\bald_eagle_11.npy:
	BEFORE: (128, 216) 
	AFTER: (128, 216)
Processed: ../data/processed\bald_eagle\bald_eagle_12.npy:
	BEFORE: (128, 216) 
	AFTER: (128, 216)
Processed: ../data/processed\bald_eagle\bald_eagle_13.npy:
	BEFORE: (128, 216) 
	AFTER: (128, 216)
Processed: ../data/processed\bald_eagle\bald_eagle_14.npy:
	BEFORE: (128, 216) 
	AFTER: (128, 216)
Processed: ../data/processed\bald_eagle\bald_eagle_15.npy:
	BEFORE: (128, 216) 
	AFTER: (128, 216)
Processed: ../data/processed\bald_eagle\bald_eagle_16.npy:
	BEFORE: (128, 216) 
	AFTER: (128, 216)
Processed: ../data/processed\bald_eagle\bald_eagle_17.npy:
	BEFORE: (128, 216) 
	AFTER: (128, 216)
Processed: ../data/processed\bald_eagle\bald_eagle_18.npy:
	BEFORE: (128, 216) 
	AFTER: (128, 216)
Processed: 